#Paquetes necesarios

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import cv2  
import math 
from ultralytics import YOLO
from collections import defaultdict
import numpy as np
import easyocr
import csv

In [ ]:
# Cargar el modelo YOLOv8 (puedes usar uno preentrenado)
model = YOLO('yolov8n.pt')  # También puedes usar 'yolov8s.pt' para un modelo más grande

# Entrenar el modelo con tu dataset personalizado
model.train(
    data='license_plates.yaml',  # El archivo de configuración YAML
    epochs=50,  # Ajusta el número de épocas según sea necesario
    imgsz=500,  # Tamaño de las imágenes (puedes ajustarlo)
    batch=15,   # Tamaño del batch
    device='0'    # Para entrenar con GPU, usa '0'. Para CPU, usa 'cpu'
)
model.val()

In [17]:
# Carga del modelo YOLO para la detección general de objetos (coches, personas, etc.)
yolo11 = YOLO('yolo11n.pt')

# Carga del modelo YOLO para detectar matrículas
license_plates_model = YOLO('C:\\Users\\Administrator\\Documents\\GitHub\\Practica-4-VC\\modelo-en-weight\\weights\\best.pt')

# Carga del modelo EasyOCR para reconocimiento de texto
reader = easyocr.Reader(['es'])

# Etiquetas de las clases del modelo de detección general
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck"]

# Captura desde el archivo de video
filename = "C0142.MP4"
vid = cv2.VideoCapture(filename)

# Propiedades del video original
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
fps = vid.get(cv2.CAP_PROP_FPS)

# Inicializamos el VideoWriter para guardar el video anotado
output_filename = 'video_anotado_solo_deteccion.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codificador de video
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

# Historial de seguimiento (opcional, puedes eliminar si no lo necesitas)
track_history = defaultdict(lambda: [])

while True:
    # Leer fotograma por fotograma
    ret, frame = vid.read()

    if not ret:
        break

    # Detección general (personas, coches, etc.) con YOLOv8
    results = yolo11.track(frame, persist=True, classes=[0, 2, 3, 5, 7])

    if results is not None:
        # Procesamos cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Coordenadas del contenedor detectado
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                #print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                #print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor para el objeto detectado (vehículo, etc.)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(frame, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)


                # Recorta el objeto de interés para buscar la matrícula
                objeto_interes = frame[y1:y2, x1:x2]
                
                # Detección de matrículas dentro del objeto de interés (vehículo)
                plate_results = license_plates_model(objeto_interes)
                
                for plate_result in plate_results:
                    plate_boxes = plate_result.boxes
                    
                    for plate_box in plate_boxes:
                        # Coordenadas de la matrícula dentro del objeto de interés
                        px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
                        
                        # Ajustar las coordenadas para el contexto global del video original
                        global_x1 = x1 + px1
                        global_y1 = y1 + py1
                        global_x2 = x1 + px2
                        global_y2 = y1 + py2
                        
                        # Dibuja el contenedor para la matrícula detectada
                        cv2.rectangle(frame, (global_x1, global_y1), (global_x2, global_y2), (0, 255, 0), 2)
                        
                        # Recorta la región de la matrícula para OCR
                        placa_img = frame[global_y1:global_y2, global_x1:global_x2]
                        
                        # Utilizar EasyOCR para leer el texto de la matrícula
                        result_ocr = reader.readtext(placa_img, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
                        
                        # Mostrar el texto detectado y dibujarlo en la imagen
                        for (bbox, text, prob) in result_ocr:
                            print(f"Texto detectado: {text} con una probabilidad de {prob:.2f}")
                            cv2.putText(frame, text, (global_x1, global_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

        # Guardar el fotograma procesado en el archivo de video
        out.write(frame)

        # Mostrar el video procesado en tiempo real
        cv2.imshow("Detección YOLO y OCR", frame)

        # Detener si se presiona la tecla "q"
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Detener el bucle si se presiona ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar recursos
vid.release()
out.release()
cv2.destroyAllWindows()


Reconocimiento de caracteres tras instalar easyocr

In [2]:
def addLineToCSV(
    frame_count: int, 
    class_name: str, 
    confidence: float, 
    track_id: int, 
    x1: int, 
    y1: int, 
    x2: int, 
    y2: int, 
    matricula: str = "", 
    m_confidence: str = "", 
    m_x1: str = "", 
    m_y1: str = "", 
    m_x2: str = "", 
    m_y2: str = "", 
    texto_matricula: str = ""
):
    with open('output.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([
            frame_count, 
            class_name, 
            confidence, 
            track_id, 
            x1, 
            y1, 
            x2, 
            y2, 
            matricula, 
            m_confidence, 
            m_x1, 
            m_y1, 
            m_x2, 
            m_y2, 
            texto_matricula
        ])

In [3]:
# Carga del modelo YOLO para detección general de objetos
yolo11 = YOLO('yolo11n.pt')

# Carga del modelo YOLO para detectar matrículas
license_plates_model = YOLO('C:\\Users\\Administrator\\Documents\\GitHub\\Practica-4-VC\\modelo-en-weight\\weights\\best.pt')

# Carga del modelo EasyOCR para reconocimiento de texto en español
reader = easyocr.Reader(['es'])

# Etiquetas de las clases del modelo de detección general
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck"]

# Captura del archivo de video
filename = "C0142.MP4"
vid = cv2.VideoCapture(filename)

# Propiedades del video original
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
fps = vid.get(cv2.CAP_PROP_FPS)

# Inicializamos el VideoWriter para guardar el video anotado
output_filename = 'video_anotado.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

frame_count = 0

while vid.isOpened():
    # Leer fotograma por fotograma
    ret, frame = vid.read()

    if not ret:
        break  # Fin del video
    
    frame_count+=1

    # Detección de objetos (vehículos, personas, etc.) con YOLO
    results = yolo11(frame, classes=[0, 2, 3, 5, 7])

    if results:
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Coordenadas del objeto detectado (vehículo)
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                #print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                #print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor para el objeto detectado (vehículo, etc.)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(frame, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

                # Recortar la región del objeto de interés (posible vehículo)
                objeto_interes = frame[y1:y2, x1:x2] 

                # Detección de matrículas dentro del vehículo
                plate_results = license_plates_model(objeto_interes)

                if not (plate_results): 
                    addLineToCSV(frame_count, classNames[cls], confidence, track_id, x1, y1, x2, y2)

                for plate_result in plate_results:
                    plate_boxes = plate_result.boxes
                    for plate_box in plate_boxes:
                        # Coordenadas de la matrícula
                        px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
                        
                        # Ajustar las coordenadas al fotograma completo
                        global_x1 = x1 + px1
                        global_y1 = y1 + py1
                        global_x2 = x1 + px2
                        global_y2 = y1 + py2

                        # Dibujar la caja alrededor de la matrícula detectada
                        cv2.rectangle(frame, (global_x1, global_y1), (global_x2, global_y2), (0, 255, 0), 2)

                        # Recortar la región de la matrícula para OCR
                        placa_img = frame[global_y1:global_y2, global_x1:global_x2]

                        # Usar EasyOCR para leer el texto de la matrícula
                        result_ocr = reader.readtext(placa_img, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')

                        if not (result_ocr): 
                            addLineToCSV(frame_count, classNames[cls], confidence, track_id, x1, y1, x2, y2, matricula=True, m_x1=global_x1, m_y1=global_y1, m_x2=global_x2, m_y2=global_y2)

                        for (bbox, text, prob) in result_ocr:
                            print(f"Texto detectado: {text} con una probabilidad de {prob:.2f}")
                            cv2.putText(frame, text, (global_x1, global_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
                            
                            addLineToCSV(frame_count, classNames[cls], confidence, track_id, x1, y1, x2, y2, matricula=True, m_x1=global_x1, m_y1=global_y1, m_x2=global_x2, m_y2=global_y2, texto_matricula=text)
                            
                            #addLineToCSV()


    # Guardar el fotograma procesado en el archivo de salida
    out.write(frame)

# Liberar recursos
vid.release()
out.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 384x640 4 cars, 1 bus, 75.1ms
Speed: 4.5ms preprocess, 75.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 448x512 (no detections), 48.0ms
Speed: 2.5ms preprocess, 48.0ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 512)

0: 416x512 (no detections), 45.0ms
Speed: 2.0ms preprocess, 45.0ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 512)

0: 320x512 (no detections), 41.5ms
Speed: 1.0ms preprocess, 41.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 512)

0: 352x512 (no detections), 51.5ms
Speed: 2.0ms preprocess, 51.5ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 512)

0: 384x512 (no detections), 52.0ms
Speed: 1.5ms preprocess, 52.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 512)

0: 384x640 4 cars, 1 bus, 55.5ms
Speed: 3.0ms preprocess, 55.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 416x512 (no detections), 38.0ms
Speed: 1.5ms preprocess, 38.0ms infer

KeyboardInterrupt: 